# Clustering


In [1]:
from os import path
import context
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
import xgboost as xgb

import sklearn
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score

import seaborn as sns
import scikitplot as skplt
import matplotlib.pyplot as plt

In [25]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

# Meta parameters
VERBOSE = 0
FOLDS = 5

show_fold_stats = False
# show_fold_stats = False # set to True if all OOF results wanted

do_plot_ROC = True # set to True to plot ROC when predicting cirrhosis

# test_train_split_SEED = 1970
test_train_split_SEED = 1971

In [3]:
def plot_ROC(fpr, tpr, m_name):
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    plt.figure(figsize=(6, 6))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc, alpha=0.5)

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('ROC for %s'%m_name, fontsize=20)
    plt.legend(loc="lower right", fontsize=16)
    plt.show()

In [4]:
pd_abundance = pd.read_csv(path.join(context.proj_dir, 'data', 'abundance.csv'))
pd_abundance = pd_abundance.loc[:,(pd_abundance!=0).sum()>10]
disease = pd_abundance.loc[:,'disease']
d_name = pd_abundance.loc[:,'dataset_name']
print(disease.value_counts())
print(d_name.value_counts())

# list of diseases we want to analyze and predict
diseases = ['obesity', 'cirrhosis', 't2d', 'cancer']

n                             2054
nd                             475
t2d                            223
obesity                        164
ibd_ulcerative_colitis         148
cirrhosis                      118
leaness                         89
stec2-positive                  52
impaired_glucose_tolerance      49
cancer                          48
n_relative                      47
y                               36
small_adenoma                   26
ibd_crohn_disease               25
 -                              20
large_adenoma                   13
overweight                      10
-                                7
obese                            5
underweight                      1
Name: disease, dtype: int64
hmp                                  762
doyle_bt2                            458
Neilsen_genome_assembly              382
Segre_Human_Skin                     291
t2dmeta_long                         290
Chatelier_gut_obesity                278
Quin_gut_liver_cirrhosis  

/home/ahmad/myenvs/python3-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,5,9,20,26,42,47,51,52,59,61,68,76,77,78,79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cols = pd_abundance.columns.tolist()
species = [x for x in cols if (x.startswith('k_') and "s__" in x)]
metadata = [x for x in cols if not x.startswith('k_')]
pd_abundance_conv = pd_abundance.copy()
pd_abundance_conv = pd_abundance_conv[species].astype('float64')
pd_abundance_conv = pd.concat([pd_abundance[metadata], pd_abundance_conv], axis = 1)

# controls/healthy samples from Human Microbiome Project coded 'hmp' and 'hmpii'.
# 't2d' stands for Type 2 Diabetes. We will combine a few studies into single dataset.
data_sets = {'control':['hmp', 'hmpii'],'t2d':['WT2D','t2dmeta_long','t2dmeta_short'], 'cirrhosis' : ['Quin_gut_liver_cirrhosis'],
             'cancer' : ['Zeller_fecal_colorectal_cancer'], 'obesity' : ['Chatelier_gut_obesity']}
# combine controls from different studies into one
pd_abundance_conv['disease'] = pd_abundance_conv['disease'].apply(lambda x: 'control' if ((x == 'n') or (x == 'nd') or (x == 'leaness')) else x)
# separate controls and diseases into 2 dataframes
pd_control = pd_abundance_conv.loc[pd_abundance_conv['disease'] == 'control']
pd_disease = pd_abundance_conv.loc[pd_abundance_conv['disease'] != 'control']

# we won't consider diseases from this list
not_disease = [d for d in pd_disease.disease.unique().tolist() if d not in diseases]
for d in not_disease:
    pd_disease = pd_disease.drop(pd_disease.loc[pd_disease['disease'] == d].index, axis = 0)

In [6]:
pd_abundance_conv.shape

(3610, 1727)

In [34]:
def disease_distinction(d, counter_d, seed):
    print('-' * 80)
    print('Discriminate %s from %s'%(d,counter_d))

    skf = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = seed)
    models = []
    oof_scores= []
    ds_names = data_sets[d]
    if len(ds_names) == 1:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[pd_disease['dataset_name'] == ds_names[0]]
    else:
        pd_cont = pd_control.loc[pd_control['dataset_name'] == ds_names[0]]
        pd_dis = pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds_names[0])]
        for ds in ds_names[1:]:
            pd_cont = pd.concat([pd_cont, pd_control.loc[pd_control['dataset_name'] == ds]], axis = 0)
            pd_dis = pd.concat([pd_dis, pd_disease.loc[(pd_disease['disease'] == d) & (pd_disease['dataset_name'] == ds)]], axis = 0)

    #     create dataset with all other diseases, target set to 0
    pd_others = pd_disease.loc[(pd_disease['disease'] == counter_d)]
    target_others = pd_others['disease'].apply(lambda x: 1 if x == d else 0)

    pd_train = pd.concat([pd_cont, pd_dis], axis = 0)
    #     adding control data from healthy subject, data by HMP
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmp']], axis = 0)
    pd_train = pd.concat([pd_train, pd_control.loc[pd_control['dataset_name'] == 'hmpii']], axis = 0)
    target = pd_train['disease']
    #     convert text target into binary
    binary_target = target.apply(lambda x: 1 if x == d else 0)
    # binary_target.value_counts()
    #     use only abundance data for training
    pd_others = pd_others[species]
    pd_train  = pd_train[species]

    #     this split provides us with preserved test set
    disease_train, disease_test, disease_y_train, disease_y_test = train_test_split(pd_train, binary_target, test_size = 0.10,
#                                                                                     stratify = binary_target,
                                                                                    random_state = test_train_split_SEED)

    #     combining preserved test set with other diseases samples
    full_test = pd.concat([disease_test, pd_others])
    full_y_test = pd.concat([disease_y_test, target_others])

    disease_y_test.value_counts()
    disease_y_train.value_counts()
    preds = np.zeros(disease_y_test.shape[0])
    full_preds = np.zeros(full_y_test.shape[0])

    for fold, (idxT,idxV) in enumerate(skf.split(disease_train, disease_y_train)):

        X_train = disease_train.iloc[idxT]
        X_val = disease_train.iloc[idxV]
        y_train = disease_y_train.iloc[idxT]
        y_val = disease_y_train.iloc[idxV]

        XGB_model = XGBClassifier(n_estimators=5000, max_depth=None,
                            learning_rate=0.05,
                            objective='binary:logistic',
                            metric='auc',
                            verbosity  = VERBOSE,
                            n_jobs=-1, random_state  = seed, silent=True)

        if show_fold_stats:
            print('-' * 80)
            print('Fold : %s'%(fold+1))

        XGB_model.fit(X_train, y_train,
                        eval_set = [(X_val, y_val)],
                        eval_metric=['logloss'],
                        early_stopping_rounds = 100, verbose=VERBOSE )

        XGB_preds = XGB_model.predict_proba(X_val)
        XGB_score = metrics.roc_auc_score(y_val, XGB_preds[:,1])
        XGB_class = XGB_model.predict(X_val)

        XGB_test = XGB_model.predict_proba(disease_test)
        XGB_test_score = metrics.roc_auc_score(disease_y_test, XGB_test[:,1])
        XGB_test_class = XGB_model.predict(disease_test)

        full_test_preds = XGB_model.predict_proba(full_test)
        full_test_score = metrics.roc_auc_score(full_y_test, full_test_preds[:,1])
        full_test_class = XGB_model.predict(full_test)

        f1s = f1_score(y_val, XGB_class)
        recall = metrics.recall_score(y_val, XGB_class)
        precision_score = metrics.precision_score(y_val, XGB_class)

        f1_test = f1_score(disease_y_test, XGB_test_class)
        recall_test = metrics.recall_score(disease_y_test, XGB_test_class)
        precision_score_test = metrics.precision_score(disease_y_test, XGB_test_class)

        f1_full_test = f1_score(full_y_test, full_test_class)
        recall_full_test = metrics.recall_score(full_y_test, full_test_class)
        precision_full_test = metrics.precision_score(full_y_test, full_test_class)

        if show_fold_stats:
            print('ROC AUC score for XGBoost model valid set: %.4f'%XGB_score)
            print('F1 score: %0.4f'%f1s)
            print(confusion_matrix(y_val, XGB_class))

            print('ROC AUC score for XGBoost model test set: %.4f'%XGB_test_score)
            print('F1 score: %0.4f'%f1_test)
            print(confusion_matrix(disease_y_test, XGB_test_class))

            print('ROC AUC score for full test set including other diseases used as false controls: %.4f'%full_test_score)
            print('F1 score: %0.4f'%f1_full_test)
            print(confusion_matrix(full_y_test, full_test_class))

        preds += XGB_test[:,1] / FOLDS
        full_preds += full_test_preds[:,1] / FOLDS

        fold_score = [XGB_score,f1s,recall,precision_score, XGB_test_score,f1_test,recall_test,precision_score_test]
        oof_scores.append({fold : fold_score})
        models.append(XGB_model)

    avg_test_score = metrics.roc_auc_score(disease_y_test, preds)
    avg_class = np.where(preds < 0.5, 0, 1)
    avg_f1_test = f1_score(disease_y_test, avg_class)
    avg_recall_test = metrics.recall_score(disease_y_test, avg_class)
    avg_precision_score_test = metrics.precision_score(disease_y_test, avg_class)

    avg_full_test_score = metrics.roc_auc_score(full_y_test, full_preds)
    avg_class_full = np.where(full_preds < 0.5, 0, 1)
    avg_f1_test_full = f1_score(full_y_test, avg_class_full)
    avg_recall_full_test = metrics.recall_score(full_y_test, avg_class_full)
    avg_precision_full_test = metrics.precision_score(full_y_test, avg_class_full)

    if show_fold_stats:
        print('-' * 80)
        print('ROC AUC score for %s averaged over %i folds: %.4f'%(d, FOLDS, avg_test_score))
        print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test, avg_recall_test, avg_precision_score_test))
        print('Confusion matrix for %s averaged across %i folds '%(d,FOLDS))
        print(confusion_matrix(disease_y_test, avg_class))


    print('ROC AUC score for %s against %s averaged over %i folds : %.4f'%(d, counter_d, FOLDS, avg_full_test_score ))
    print('F1 : %.4f, Recall : %.4f , Precision : %.4f'%(avg_f1_test_full, avg_recall_full_test, avg_precision_full_test))
    print('Confusion matrix for %s against %s averaged across %i folds'%(d, counter_d, FOLDS))
    print(confusion_matrix(full_y_test, avg_class_full))

    # if do_plot_ROC:
    #     (fpr, tpr, thresholds) = metrics.roc_curve(disease_y_test, preds)
    #     plot_ROC(fpr, tpr, d)

    return full_preds, full_y_test, \
           {d : (avg_test_score, avg_f1_test, avg_recall_test, avg_precision_score_test, oof_scores)}, \
           models

In [35]:
full_predictions = []
full_test_y = []
other_metrics = []
skf_models = []
SEED = 1970
print('=' * 80)
print('Seed : %i'%SEED)
d = 'cirrhosis'
for counter in diseases:
    if counter != d:
        dc_pred, y_preds, other, model = disease_distinction(d, counter, SEED)
        full_predictions.append(dc_pred)
        full_test_y.append(y_preds)
        other_metrics.append(other)
        skf_models.append(model)

Seed : 1970
--------------------------------------------------------------------------------
Discriminate cirrhosis from obesity
ROC AUC score for cirrhosis against obesity averaged over 5 folds : 1.0000
F1 : 1.0000, Recall : 1.0000 , Precision : 1.0000
Confusion matrix for cirrhosis against obesity averaged across 5 folds
[[279   0]
 [  0   7]]
--------------------------------------------------------------------------------
Discriminate cirrhosis from t2d
ROC AUC score for cirrhosis against t2d averaged over 5 folds : 0.9996
F1 : 0.5600, Recall : 1.0000 , Precision : 0.3889
Confusion matrix for cirrhosis against t2d averaged across 5 folds
[[327  11]
 [  0   7]]
--------------------------------------------------------------------------------
Discriminate cirrhosis from cancer
ROC AUC score for cirrhosis against cancer averaged over 5 folds : 1.0000
F1 : 0.8235, Recall : 1.0000 , Precision : 0.7000
Confusion matrix for cirrhosis against cancer averaged across 5 folds
[[160   3]
 [  0  

In [16]:
other_disease_list = [dis for dis in diseases if dis != d]
feature_list = []
for i in range(3):
    skf_model = skf_models[i][0]
    imp_values = skf_model.get_booster().get_fscore()
    for k,v in imp_values.items():
        feature_list.append([other_disease_list[i], k,v])
df_feature_imp = pd.DataFrame(feature_list, columns=['disc_disease', 'feature', 'value'])
df_feature_imp.to_csv('../data_processed/cirrhosis_feature_importance.csv', index=False)

['obesity', 't2d', 'cancer']